In [1]:
import os, time, datetime, sched, shutil
import pandas as pd
import openpyxl

DIR_PATH = str(os.getcwd())+'\\images'

In [2]:

# Checking if the output.xlsx file exist
if "output.xlsx" in os.listdir(os.getcwd()):
    sheet1 = pd.read_excel("output.xlsx", index_col=0, sheet_name="Sheet1" ,engine='openpyxl')
    sheet2 = pd.read_excel("output.xlsx", index_col=0, sheet_name="Sheet2" ,engine='openpyxl')
# if not create one
else:
    print("There is no output.xlsx file in the working directory, creating one...")
    sheet1 = ([])
    sheet2 = ([])

In [3]:
""" 
    Inputs:
        start: start time of an interval (string) (hh:mm:ss)
        created: created time of an image (string) (hh:mm:ss)
        end: end time of an interval (string) (hh:mm:ss)
        
    Returns:
        true if the created time is between the time interval, otherwise returns false
"""
def compareTime(start, created, end, currentDate, createdDate):    
    now = datetime.datetime.now()
    start = datetime.datetime.strptime(start, "%H:%M:%S")
    start = now.replace(hour=start.time().hour, minute=start.time().minute, second=start.time().second, microsecond=0)

    created = datetime.datetime.strptime(created, "%H:%M:%S")
    created = now.replace(
                          hour=created.time().hour, 
                          minute=created.time().minute, 
                          second=created.time().second, microsecond=0)

    end = datetime.datetime.strptime(end, "%H:%M:%S")
    end = now.replace(hour=end.time().hour, minute=end.time().minute, second=end.time().second, microsecond=0)
    
    if start < created < end and createdDate == currentDate:
        return True
    
    return False


def move_images(dir_path, target_dir, start, end, currentDate, sheet1, sheet2):
    # Interval images counter
    print(currentDate)
    num_images = 0
    xlsx_changed = False
    for img in os.listdir(dir_path):
        img_path = dir_path+'\\'+img
        
        timeStamp = time.ctime(os.path.getctime(img_path)).split()
        
        createdDate = timeStamp[1] + '-' + timeStamp[2] + '-' + timeStamp[4]
        createdDate = time.strftime("%Y-%m-%d", time.strptime(createdDate, "%b-%d-%Y")) 
        
        createdTime = timeStamp[3]
        
        if compareTime(start, createdTime, end, currentDate, createdDate):
            if not os.path.isdir(target_dir):
                os.mkdir(target_dir)
            
            xlsx_changed = True
            num_images = num_images+1 
            # Append new image to sheet 1
            sheet1 = sheet1.append({'createdTimeStamp': createdDate + " " + createdTime, 'location': img_path} , ignore_index=True)
            shutil.move(img_path, target_dir)
    
    if xlsx_changed:
        # Append number of images to sheet 2
        sheet2 = sheet2.append({'dateTime': currentDate + " " + start + "-" + end, 'numImages': num_images} , ignore_index=True)

        try:
            # Write changes
            with pd.ExcelWriter('output.xlsx') as writer:  
                sheet1.to_excel(writer, sheet_name='Sheet1')
                sheet2.to_excel(writer, sheet_name='Sheet2')
        except:
            print('The xlsx file is open, saving changes in the next time interval')
    
    return sheet1, sheet2

s = sched.scheduler(time.time, time.sleep)

"""
    Function that runs every 30 minutes and creates a directory with the current timestamp
"""
def repeat(sc, sheet1, sheet2):
    print('running')
    print(sheet1)
    start = datetime.datetime.now().strftime('%H:%M:%S').split()[0]
    currentDate = datetime.datetime.now().strftime('%Y-%m-%d').split()[0]
    
    start_list = start.split(':')
    hours = start_list[0]
    mins = start_list[1]
    secs = start_list[2]
    
    totalSecs = ((int(hours) * 60 + int(mins)) * 60 + int(secs)) - (30)
    totalSecs, secs = divmod(totalSecs, 60)
    hours, mins = divmod(totalSecs, 60)
    start = str(hours)+':'+str(mins)+':'+str(secs)

    totalSecs = ((int(hours) * 60 + int(mins)) * 60 + int(secs)) + (30)
    totalSecs, sec = divmod(totalSecs, 60)
    hr, min = divmod(totalSecs, 60)
    
    end = str(hr)+':'+str(min)+':'+str(sec)

    target_dir = currentDate + ' ' + str(hours)+'_'+str(mins)+'_'+str(secs)+'-'+str(hr)+'_'+str(min)+'_'+str(sec)
    
    target_dir = str(os.getcwd()) + '\\' + target_dir
    
    changed_sheet1, changed_sheet2 = move_images(DIR_PATH, target_dir, start, end, currentDate, sheet1, sheet2)
    
    sheet1 = changed_sheet1
    sheet2 = changed_sheet2

    
    s.enter(30, 1, repeat, (sc,sheet1, sheet2))

In [ ]:
s.enter(0, 1, repeat, (s, sheet1, sheet2))
s.run()

running
Empty DataFrame
Columns: []
Index: []
2021-08-23
running
Empty DataFrame
Columns: []
Index: []
2021-08-23
running
      createdTimeStamp                                           location
0  2021-08-23 12:23:25  C:\Users\te455434\OneDrive - TE Connectivity\D...
1  2021-08-23 12:23:25  C:\Users\te455434\OneDrive - TE Connectivity\D...
2  2021-08-23 12:23:25  C:\Users\te455434\OneDrive - TE Connectivity\D...
3  2021-08-23 12:23:25  C:\Users\te455434\OneDrive - TE Connectivity\D...
2021-08-23
running
      createdTimeStamp                                           location
0  2021-08-23 12:23:25  C:\Users\te455434\OneDrive - TE Connectivity\D...
1  2021-08-23 12:23:25  C:\Users\te455434\OneDrive - TE Connectivity\D...
2  2021-08-23 12:23:25  C:\Users\te455434\OneDrive - TE Connectivity\D...
3  2021-08-23 12:23:25  C:\Users\te455434\OneDrive - TE Connectivity\D...
2021-08-23
running
      createdTimeStamp                                           location
0  2021-08-23 12:23:25  C: